In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.seasonal import seasonal_decompose
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Función para cargar los datos
def load_data(data_string):
    lines = data_string.strip().split('\n')
    data = []
    for line in lines:
        # Extraer fecha y valor
        if line.startswith('**SKU_25'):
            continue
        fecha = line[:10]
        valor = int(line[10:])
        data.append([fecha, valor])
    # Crear DataFrame
    df = pd.DataFrame(data, columns=['Fecha', 'Valor'])
    df['Fecha'] = pd.to_datetime(df['Fecha'])
    df.set_index('Fecha', inplace=True)
    return df

In [ ]:
# Función modificada para ejecutar prueba Ljung-Box
def run_ljung_box_test(df, lags=30):
    """
    Ejecuta la prueba Ljung-Box en la serie temporal
    
    Args:
        df: DataFrame con la serie temporal
        lags: Número de rezagos para la prueba
        
    Returns:
        DataFrame con los estadísticos y p-values
    """
    if len(df) > lags:
        result = acorr_ljungbox(df, lags=lags)
        lb_df = pd.DataFrame({
            'lag': range(1, lags + 1),
            'lb_stat': result[0],
            'lb_pvalue': result[1]
        })
    else:
        # Si no hay suficientes datos, ajustamos los lags
        max_lags = len(df) - 1
        if max_lags > 0:
            result = acorr_ljungbox(df, lags=max_lags)
            lb_df = pd.DataFrame({
                'lag': range(1, max_lags + 1),
                'lb_stat': result[0],
                'lb_pvalue': result[1]
            })
        else:
            lb_df = pd.DataFrame({
                'lag': [1],
                'lb_stat': [np.nan],
                'lb_pvalue': [1.0]
            })
    
    return lb_df

In [ ]:
# Función para verificar estacionariedad y tipo de descomposición adecuada
def check_decomposition_type(df):
    # Analizar por ventanas temporales
    window_size = 12  # Para datos mensuales
    df_reset = df.reset_index()
    df_reset['window'] = df_reset.index // window_size
    window_stats = df_reset.groupby('window')['Valor'].agg(['mean', 'std'])
    
    # Calcular correlación entre media y desviación estándar
    correlation = np.corrcoef(window_stats['mean'], window_stats['std'])[0,1]
    
    # Determinar tipo de descomposición
    if correlation > 0.5:
        decomp_type = 'multiplicative'
        print(f"Correlación entre media y desv. estándar: {correlation:.4f}. Recomendación: modelo multiplicativo")
    else:
        decomp_type = 'additive'
        print(f"Correlación entre media y desv. estándar: {correlation:.4f}. Recomendación: modelo aditivo")
    
    return decomp_type

In [ ]:
# Función para visualizar serie y componentes
def plot_series_components(df, decomp_type):
    # Realizar descomposición
    decomposition = seasonal_decompose(df['Valor'], model=decomp_type, period=12)
    
    # Visualizar componentes
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(14, 16))
    
    # Serie original
    ax1.plot(df.index, df['Valor'])
    ax1.set_title('Serie Original')
    ax1.set_xlabel('')
    
    # Tendencia
    ax2.plot(decomposition.trend)
    ax2.set_title('Tendencia')
    ax2.set_xlabel('')
    
    # Estacionalidad
    ax3.plot(decomposition.seasonal)
    ax3.set_title('Estacionalidad')
    ax3.set_xlabel('')
    
    # Residuos
    ax4.plot(decomposition.resid)
    ax4.set_title('Residuos')
    
    plt.tight_layout()
    return decomposition

In [ ]:
# Función principal para análisis
def analyze_time_series(data_string):
    # Cargar datos
    df = load_data(data_string)
    print(f"Serie cargada con {len(df)} observaciones")
    
    # Verificar tipo de descomposición apropiada
    decomp_type = check_decomposition_type(df)
    
    # Realizar descomposición y visualizar componentes
    decomposition = plot_series_components(df, decomp_type)
    
    # Ejecutar prueba Ljung-Box en los residuos
    residuos = decomposition.resid.dropna()
    lb_results = run_ljung_box_test(residuos)
    
    # Verificar autocorrelación
    significant_lags = lb_results[lb_results['lb_pvalue'] < 0.05]
    if len(significant_lags) > 0:
        print(f"\nDetectada autocorrelación significativa en {len(significant_lags)} rezagos")
        print("Los primeros 5 rezagos significativos:")
        print(significant_lags.head())
    else:
        print("\nNo se detectó autocorrelación significativa en los residuos")
    
    # Verificar heteroscedasticidad
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.scatter(decomposition.trend.dropna(), abs(decomposition.resid.dropna()))
    ax.set_xlabel('Tendencia')
    ax.set_ylabel('Residuos Absolutos')
    ax.set_title('Diagnóstico de Heteroscedasticidad')
    plt.show()
    
    return df, decomposition, lb_results

In [ ]:
df, decomposition, lb_results = analyze_time_series(data_string)